In [1]:
import pandas as pd
from utils_analysis import prepare_scores
import matplotlib.pyplot as plt
import numpy as np

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/mila/f/floresl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/mila/f/floresl/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/mila/f/floresl/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
import json

temp = json.load(
    open("results/bart/r1/bart-base_data_flores_checkpoint-300_flores.json", "r")
)

In [3]:
df = pd.read_csv("data/flores/test.csv")

# "results/bart/r1/bart-base_data_flores_checkpoint-300_flores.json",
# "results/bart/r2/bart-base_data_flores_checkpoint-300_flores.json",
# "results/bart/r3/bart-base_data_flores_checkpoint-1000_flores.json",
# "results/t5/r1/flan-t5-base_data_flores_checkpoint-300_flores.json",
# "results/t5/r2/flan-t5-base_data_flores_checkpoint-300_flores.json",
# "results/t5/r3/flan-t5-base_data_flores_checkpoint-1000_flores.json",

df_scores, beam_search_results = prepare_scores(
    "results/t5/r1/flan-t5-base_data_flores_checkpoint-300_flores.json",
    df["target"],
    "bleu",
)

float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero


/home/mila/f/floresl/beam-search/utils_analysis.py:37: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  abs(spearmanr(ground_truth_score, confidence_score_dict[str(k)]).statistic)  # type: ignore


In [4]:
df_scores.drop(["sentences", "dropout_sentences"], axis=1).corr(method="spearman")[
    "bleu"
]

length_normalized_log_probs    0.162446
mean_token_entropy            -0.075314
dropout_bleu_variance          0.367318
dropout_meteor_score           0.393195
dropout_entropy               -0.168239
dropout_disagreement           0.113432
bleu                           1.000000
beam_score_ratios_99           0.104255
beam_score_log_probs_99        0.130958
beam_score_top_k_99            0.150949
beam_score_impt_wt_99         -0.151252
Name: bleu, dtype: float64

In [5]:
beam_score_log_probs = pd.DataFrame(beam_search_results["beam_score_log_probs"])
worst_rouge_idxs = df_scores.sort_values(by="bleu").index[:100]
best_rouge_idxs = df_scores.sort_values(by="bleu").index[::-1][:100]

In [6]:
from utils_tail_probs import softmax, tail_index
from scipy.stats import spearmanr

In [ ]:
tail_indices = []
for i in range(len(beam_score_log_probs)):
    probs = softmax(beam_score_log_probs.iloc[i].to_numpy(), temperature=1)
    tail_indices.append(tail_index(probs))

spearmanr(tail_indices, df_scores["bleu"])

SignificanceResult(statistic=0.13951295732345192, pvalue=8.40907396480753e-06)

In [10]:
import json

baseline_probs = json.load(
    open(
        "/home/mila/f/floresl/beam-search/results/t5_baseline/google_flan-t5-base_flores.json",
        "r",
    )
)

In [11]:
baseline_log_probs = baseline_probs["beam_score_log_probs"]
baseline_log_probs = pd.DataFrame(baseline_log_probs).values

beam_score_log_probs = beam_score_log_probs.values

In [12]:
from scipy.spatial.distance import jensenshannon

In [13]:
js_distances = []
for i in range(1012):
    js = jensenshannon(
        softmax(beam_score_log_probs[i], temperature=1),
        softmax(baseline_log_probs[i], temperature=1),
    )
    js_distances.append(js)
spearmanr(js_distances, df_scores["bleu"])

SignificanceResult(statistic=0.12106860235911387, pvalue=0.00011298028361375931)

In [14]:
js_distances = []
for i in range(1012):
    js = jensenshannon(
        softmax(beam_score_log_probs[i], temperature=1), np.array([1 / 100] * 100)
    )
    js_distances.append(js)
spearmanr(js_distances, df_scores["bleu"])

SignificanceResult(statistic=0.13986105130422422, pvalue=7.979991608000934e-06)